In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from rnn_base import RNN
from history_rnn import HistoryRNN
from op_selector import OpSel
from mem_selector import MemSel
from ops import Operations
from session import *
from data_gen import samples_generator, split_train_test
import pickle
from data_gen import samples_generator, split_train_test

modelName = 'RNN'
func = 'np_avg_val-2ops'
name = 'state_size_400-num_samples_2500-batch_size_100-learning_rate_0.001-epsilon_0.001-loss_weight_0.5-num_features_10-norm_True-grad_norm_1000.0-softmax_sat_100-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-seed_167'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 0
timestep 1
[<tf.Variable 'RNN_mem/Params/W_mem:0' shape=(410, 400) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(400,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W2_mem:0' shape=(400, 3) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(3,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W:0' shape=(410, 400) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(400,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W2:0' shape=(400, 3) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(3,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_0:0' shape=(410, 400) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_1:0' shape=(400,) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_2:0' shape=(400, 3) dtype=float64>, <tf.Tensor 'RNN_op/Gr

In [2]:
def print_errors(error_lst, name):
    print("")
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
       

In [4]:
def print_ops_matrix(total_error, matrix_lst, ops_list):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
            

In [5]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")


sofmax_train_error[0] err is 8.33676040529e-15
sofmax_train_error[1] err is 7.43772168762e-15
sofmax_train_error[2] err is 7.00504864831e-15
sofmax_train_error[3] err is 7.88633124267e-15
sofmax_train_error[4] err is 9.95716362509e-15
sofmax_train_error[5] err is 7.23188395829e-15
sofmax_train_error[6] err is 8.2318619949e-15
sofmax_train_error[7] err is 7.08335739441e-15
sofmax_train_error[8] err is 8.38248785052e-15
sofmax_train_error[9] err is 7.56101443111e-15
sofmax_train_error[10] err is 8.80295592837e-15
sofmax_train_error[11] err is 7.34058004783e-15
sofmax_train_error[12] err is 6.93936895823e-15
sofmax_train_error[13] err is 9.04688667878e-15
sofmax_train_error[14] err is 8.37243188833e-15
sofmax_train_error[15] err is 7.86166798323e-15

hardmax_train_error[0] err is 2.92052638999e-28
hardmax_train_error[1] err is 1.56662845396e-28
hardmax_train_error[2] err is 2.10415164958e-28
hardmax_train_error[3] err is 2.58600252996e-28
hardmax_train_error[4] err is 3.55126459491e-28
h

In [6]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train"][1])
print_ops_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train"][1], cfg["used_ops_obj"])

Error for this matrix is 7.43772168762e-15

##Elem-0--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-1--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-2--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-3--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-4--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-5--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-6--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-7--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-8--#############################################################
[ tf_add ] [ tf_divide ] 
##Elem-9--#############################################################
[ 

In [7]:
#print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
print_ops_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1], cfg["used_ops_obj"])

Error for this matrix is 7.43772168762e-15

##Elem-0--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-1--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-2--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-3--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-4--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-5--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-6--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-7--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-8--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-9--#

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_train_mem"][1], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_traind_train"][1], res["outputs_traind_train_mem"][1])

In [ ]:
res["batchesX_train"][1]

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

In [ ]:
res - y_test[0:100,]

In [ ]:
y_test[0:100,]

In [ ]:
res

In [ ]:
len(res["softmaxes_traind_train"])